# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtest.utils import run_strategy, run_wfo, walk_forward


import numpy as np
from sklearn.linear_model import LinearRegression
from backtesting import Backtest
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random

random.seed(42)

In [3]:
initial_cash = 10_000
margin = 1/30
commission = 7e-4

In [4]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()

groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [5]:
ticker = 'US500m'

print(ticker)

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2021, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    mt5.TIMEFRAME_H2,
]

symbols = get_data([ticker], intervals, date_from, date_to)

US500m
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
US500m


# Analisis preliminar

In [6]:
from backtesting import Strategy
import talib as ta
from backbone.utils.general_purpose import calculate_units_size, diff_pips

class BPercent(Strategy):
    pip_value = None
    minimum_lot = None
    contract_volume = None
    minimum_units = None
        
    risk= 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.95
    b_close_threshold = 0.5
    atr_multiplier = 1.8
    pip_value = 0.1

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )
        
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        
        self.minimum_units = self.minimum_lot * self.contract_volume
        

    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:
            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma[-1]:

                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                if units < self.minimum_units:
                    units = self.minimum_units
                
                self.buy(
                    size=units,
                    # sl=sl_price
                )
                
            if b_percent >= self.b_open_threshold and actual_close < self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                if units < self.minimum_units:
                    units = self.minimum_units

                self.sell(
                    size=units,
                    # sl=sl_price    
                )

In [7]:
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 

prices = symbols['US500m'][16386]

display(prices)

symbol_info = mt5.symbol_info(ticker)
minimum_lot = symbol_info.volume_min
contract_volume = symbol_info.trade_contract_size
pip_value = symbol_info.point

print('minimum_lot:', minimum_lot)
print('contract_volume:', contract_volume)
print('pip_value:', pip_value)

prices = prices * contract_volume * pip_value

prices



,Open,High,Low,Close,Volume,spread,real_volume
Date,,,,,,,
2021-10-27 16:00:00+00:00,4581.26,4583.41,4580.60,4583.03,214,150,0
2021-10-28 08:00:00+00:00,4560.35,4561.80,4558.61,4561.61,507,151,0
2021-10-28 10:00:00+00:00,4561.39,4570.04,4559.86,4565.60,1225,150,0
2021-10-28 12:00:00+00:00,4565.81,4583.48,4564.06,4581.75,2833,153,0
2021-10-28 14:00:00+00:00,4582.07,4593.79,4574.74,4584.57,2412,154,0
...,...,...,...,...,...,...,...
2024-08-30 10:00:00+00:00,5622.10,5625.37,5614.72,5620.28,1202,159,0
2024-08-30 12:00:00+00:00,5620.19,5630.80,5613.71,5628.63,3892,158,0
2024-08-30 14:00:00+00:00,5628.62,5634.04,5592.44,5602.81,6609,157,0


minimum_lot: 0.03
contract_volume: 1.0
pip_value: 0.01


,Open,High,Low,Close,Volume,spread,real_volume
Date,,,,,,,
2021-10-27 16:00:00+00:00,45.8126,45.8341,45.8060,45.8303,2.14,1.50,0.0
2021-10-28 08:00:00+00:00,45.6035,45.6180,45.5861,45.6161,5.07,1.51,0.0
2021-10-28 10:00:00+00:00,45.6139,45.7004,45.5986,45.6560,12.25,1.50,0.0
2021-10-28 12:00:00+00:00,45.6581,45.8348,45.6406,45.8175,28.33,1.53,0.0
2021-10-28 14:00:00+00:00,45.8207,45.9379,45.7474,45.8457,24.12,1.54,0.0
...,...,...,...,...,...,...,...
2024-08-30 10:00:00+00:00,56.2210,56.2537,56.1472,56.2028,12.02,1.59,0.0
2024-08-30 12:00:00+00:00,56.2019,56.3080,56.1371,56.2863,38.92,1.58,0.0
2024-08-30 14:00:00+00:00,56.2862,56.3404,55.9244,56.0281,66.09,1.57,0.0


In [8]:
start_date_train = Timestamp('2022-07-03 00:00:00', tz='UTC')

print(start_date_train)

2022-07-03 00:00:00+00:00


In [9]:
bt_train = Backtest(
    prices.loc[start_date_train:], 
    BPercent,
    commission=commission,
    cash=initial_cash, 
    margin=margin
)

stats = bt_train.run(
    pip_value = pip_value,
    minimum_lot = minimum_lot,
    contract_volume = contract_volume,
)

stats

Start                     2022-07-03 22:00...
End                       2024-08-30 18:00...
Duration                    788 days 20:00:00
Exposure Time [%]                   23.191981
Equity Final [$]                 13500.457268
Equity Peak [$]                  13911.651599
Return [%]                          35.004573
Buy & Hold Return [%]               48.149567
Return (Ann.) [%]                   11.894322
Volatility (Ann.) [%]               11.748477
Sharpe Ratio                         1.012414
Sortino Ratio                        1.842485
Calmar Ratio                         1.029554
Max. Drawdown [%]                  -11.552885
Avg. Drawdown [%]                   -1.306372
Max. Drawdown Duration      177 days 02:00:00
Avg. Drawdown Duration       10 days 18:00:00
# Trades                                   60
Win Rate [%]                        76.666667
Best Trade [%]                       2.088081
Worst Trade [%]                     -5.192793
Avg. Trade [%]                    

In [10]:
bt_train.plot(filename=f'./plots/{ticker}.html', resample=False)


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1614', ...)

# WFO

In [11]:
from backtest.utils import get_wfo_stats
from unittest.mock import patch


def walk_forwardd(
        strategy,
        data_full,
        warmup_bars,
        lookback_bars=28*1440,
        validation_bars=7*1440,
        params=None,
        cash=15_000, 
        commission=0.0002,
        margin=1/30,
        verbose=False
):

    stats_master = []
    equity_final = None

    # Iniciar el índice en el final del primer lookback
    i = lookback_bars + warmup_bars  

    while i < len(data_full):

        # Definir el periodo de entrenamiento correctamente
        train_data = data_full.iloc[i - lookback_bars - warmup_bars: i]
        
        if verbose:
            print(f'train from {train_data.index[0]} to {train_data.index[-1]}')

        bt_training = Backtest(
            train_data, 
            strategy, 
            cash=cash, 
            commission=commission, 
            margin=margin
        )
        
        # Sobrescribir _tqdm para eliminar la barra de progreso
        with patch('backtesting.backtesting._tqdm', lambda *args, **kwargs: args[0]):
            stats_training = bt_training.optimize(
                **params
            )
        
        # Ajustar el tamaño del periodo de validación al final del dataset
        remaining_bars = len(data_full) - i
        current_validation_bars = min(validation_bars, remaining_bars)

        # El período de validación empieza justo al final del entrenamiento
        validation_data = data_full.iloc[i - warmup_bars: i + current_validation_bars]

        if verbose:
            print(f'validate from {validation_data.index[warmup_bars]} to {validation_data.index[-1]}')

        bt_validation = Backtest(
            validation_data, 
            strategy, 
            cash=cash if equity_final is None else equity_final, 
            commission=commission, 
            margin=margin
        )
                
        validation_params = {param: getattr(stats_training._strategy, param) for param in params.keys() if param != 'maximize'}
        
        if verbose:
            print(validation_params)

        stats_validation = bt_validation.run(
            **validation_params
        )     
        
        # bt_validation.plot(filename='./plots/bug.html')
        
        equity_final = stats_validation['Equity Final [$]']
        
        if verbose:
            print(f'equity final: {equity_final}')
            print('=' * 32)

        stats_master.append(stats_validation)

        # Mover el índice `i` al final del período de validación actual
        i += current_validation_bars
        
  
    
    wfo_stats = get_wfo_stats(stats_master, warmup_bars, data_full)
    
    return wfo_stats

In [45]:
from backtest.utils import optim_func_2

strategy = BPercent
lookback_bars = 1800
validation_bars = 800
warmup_bars = 200

all_wfo_performances = pd.DataFrame()
stats_per_symbol = {}

params = {
    'pip_value': [pip_value],
    'minimum_lot': [minimum_lot],
    'contract_volume': [contract_volume],
    'b_open_threshold' : [0.9, 0.95, 1, 1.5],
    'b_close_threshold': [0.5, 0.6, 0.8, 0.9],
    'maximize':optim_func_2

}

wfo_stats = walk_forwardd(
        BPercent,
        symbols['US500m'][16386],
        warmup_bars,
        lookback_bars,
        validation_bars,
        params=params,
        cash=initial_cash, 
        commission=commission,
        margin=margin,
        verbose=True
)

train from 2021-10-27 16:00:00+00:00 to 2022-07-01 18:00:00+00:00
validate from 2022-07-03 22:00:00+00:00 to 2022-10-12 16:00:00+00:00
{'pip_value': 0.01, 'minimum_lot': 0.03, 'contract_volume': 1.0, 'b_open_threshold': 1.0, 'b_close_threshold': 0.9}
equity final: 10540.013380000002
train from 2022-02-02 00:00:00+00:00 to 2022-10-12 16:00:00+00:00
validate from 2022-10-12 18:00:00+00:00 to 2023-01-19 04:00:00+00:00
{'pip_value': 0.01, 'minimum_lot': 0.03, 'contract_volume': 1.0, 'b_open_threshold': 1.0, 'b_close_threshold': 0.9}
equity final: 11832.550234000002
train from 2022-05-12 08:00:00+00:00 to 2023-01-19 04:00:00+00:00
validate from 2023-01-19 06:00:00+00:00 to 2023-04-27 00:00:00+00:00
{'pip_value': 0.01, 'minimum_lot': 0.03, 'contract_volume': 1.0, 'b_open_threshold': 1.0, 'b_close_threshold': 0.9}
equity final: 12621.490183000013
train from 2022-08-23 08:00:00+00:00 to 2023-04-27 00:00:00+00:00
validate from 2023-04-27 02:00:00+00:00 to 2023-08-03 12:00:00+00:00
{'pip_value':

In [46]:
import plotly.graph_objects as go


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=wfo_stats._equity.index, y=wfo_stats._equity.Equity,
                    name='wfo',
                    mode='lines'))

fig.add_trace(go.Scatter(x=stats._equity_curve.index, y=stats._equity_curve.Equity,
                    name='no wfo',
                    mode='lines'))

fig.show()

In [15]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,337,296,304,41.301591,41.9291,211.470570,0.015193,2022-08-09 22:00:00+00:00,2022-08-10 14:00:00+00:00,0 days 16:00:00
1,390,379,422,42.541058,41.8242,-279.574585,-0.016851,2022-08-19 10:00:00+00:00,2022-08-25 08:00:00+00:00,5 days 22:00:00
2,-232,526,567,39.801320,40.1340,-77.181862,-0.008359,2022-09-07 22:00:00+00:00,2022-09-13 14:00:00+00:00,5 days 16:00:00
3,-200,727,765,37.059840,36.6563,80.707988,0.010889,2022-10-04 04:00:00+00:00,2022-10-07 14:00:00+00:00,3 days 10:00:00
4,-141,812,821,36.671112,36.0779,83.642926,0.016177,2022-10-13 22:00:00+00:00,2022-10-14 16:00:00+00:00,0 days 18:00:00
5,-184,831,858,36.691798,36.7649,-13.450814,-0.001992,2022-10-17 14:00:00+00:00,2022-10-20 02:00:00+00:00,2 days 12:00:00
6,216,966,999,37.590895,38.0444,97.957032,0.012064,2022-11-02 22:00:00+00:00,2022-11-07 20:00:00+00:00,4 days 22:00:00
7,318,1090,1103,39.580887,39.7430,51.551858,0.004096,2022-11-17 10:00:00+00:00,2022-11-18 12:00:00+00:00,1 days 02:00:00
8,466,1176,1201,39.865887,40.3874,243.025244,0.013082,2022-11-28 18:00:00+00:00,2022-11-30 20:00:00+00:00,2 days 02:00:00
9,-397,1297,1327,39.703488,39.7805,-30.573728,-0.001940,2022-12-12 20:00:00+00:00,2022-12-15 08:00:00+00:00,2 days 12:00:00


In [16]:
wfo_stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,276,306,3824.565323,3836.67,24.209354,0.003165,2022-07-12 22:00:00+00:00,2022-07-15 14:00:00+00:00,2 days 16:00:00
1,3,384,394,3920.572481,3965.82,135.742557,0.011541,2022-07-26 16:00:00+00:00,2022-07-27 14:00:00+00:00,0 days 22:00:00
2,3,496,504,4130.159089,4192.91,188.252733,0.015193,2022-08-09 22:00:00+00:00,2022-08-10 14:00:00+00:00,0 days 16:00:00
3,4,579,622,4254.105791,4182.42,-286.743164,-0.016851,2022-08-19 10:00:00+00:00,2022-08-25 08:00:00+00:00,5 days 22:00:00
4,-2,726,767,3980.131956,4013.40,-66.536088,-0.008359,2022-09-07 22:00:00+00:00,2022-09-13 14:00:00+00:00,5 days 16:00:00
...,...,...,...,...,...,...,...,...,...,...
57,6,6175,6179,5464.772665,5482.25,104.864010,0.003198,2024-07-02 08:00:00+00:00,2024-07-02 16:00:00+00:00,0 days 08:00:00
58,5,6307,6350,5598.105933,5576.05,-110.279665,-0.003940,2024-07-17 20:00:00+00:00,2024-07-23 12:00:00+00:00,5 days 16:00:00
59,-4,6419,6435,5485.867203,5466.70,76.668812,0.003494,2024-07-31 08:00:00+00:00,2024-08-01 16:00:00+00:00,1 days 08:00:00
60,-2,6495,6612,5316.096126,5592.15,-552.107748,-0.051928,2024-08-08 18:00:00+00:00,2024-08-22 16:00:00+00:00,13 days 22:00:00
